In [ ]:

import sqlite3
import pandas as pd
import os
import yfinance as yf
from datetime import datetime, timedelta
import time

# Suppress generated warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [218]:
def log_progress(message, ticker, log_file = "./code_log.txt"):
    """This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing"""

    timestamp_format = "%Y-%h-%d-%H:%M:%S" # Year-Monthname-Day-Hour-Minute-Second
    
    now = datetime.now() # get current timestamp
    
    timestamp = now.strftime(timestamp_format)
    
    with open(log_file, "a") as f:
        
        f.write(timestamp + " : " + message + " : " + ticker + "\n")

In [219]:
def extract(url):
    """ This function aims to extract the required
    information from the website and save it to a dataframe. The
    function yields the dataframe for further processing. """
    
    try:

        df = pd.read_html(url)[0]
    
    except Exception as e:
    
        log_progress(e, '')
        
    # This was done because of the symbols generated by wikipedia are not compatiable with Yahoo Finance.
    df['Ticker']= df['Symbol'].str.replace('.','-')
    
    # Only return needed fields.
    return df[['Ticker', 'Security', 'GICS Sector', 'GICS Sub-Industry']]

In [ ]:
def create_database(folder_name, database_name):
    
    try:
        
        if os.path.isdir(folder_name) != True:
    
            os.mkdir(folder_name)

        conn = sqlite3.connect(f'./{folder_name}/{database_name}.db')
        
    except Exception as e:
        
        log_progress(str(e), '')
        
    return conn

In [ ]:
def create_tables(conn):
    
    try:
    
        conn.execute("""CREATE TABLE IF NOT EXISTS equity (Volume, Close, Open, Low, High, AdjClose, Date, Ticker, GICSSector, GICSSectorSub, CompanyName)""")
    
    except Exception as e:
        
        log_progress(str(e), '')

In [ ]:
def collect_data(df, start_date, end_date):

    stock_list = df['Ticker'].to_list()

    data = yf.download(stock_list, start = start_date, end = end_date)

    final = pd.DataFrame()

    for row in df.iterrows():
        
        ticker = row[1]['Ticker']

        ma = []
        
        for columns in ['Volume', 'Close', 'Open', 'Low', 'High', 'Adj Close']:

            cars = data[columns][ticker].reset_index()
            
            ma.append(data[columns][ticker].to_list())
            
        ma.append(cars['Date'].to_list())
        
        company_data = pd.DataFrame(ma).T
        
        company_data['Ticker'] = ticker
        
        company_data['GICSSector'] = row[1]['GICS Sector']
                
        company_data['GICSSectorSub'] = row[1]['GICS Sub-Industry']
                
        company_data['CompanyName'] = row[1]['Security']
        
        final = pd.concat([final, company_data])
        
    final.columns = ['Volume', 'Close', 'Open', 'Low', 'High', 'AdjClose', 'Date', 'Ticker', 'GICSSector', 'GICSSectorSub', 'CompanyName']

    final.reset_index(drop = True)
    
    final['Date'] = pd.to_datetime(final['Date'])
    
    return final

In [233]:
def upload(data, conn):
    
    data.to_sql('equity', conn, if_exists = 'append', index = False)

In [234]:
df = extract('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

conn = create_database('data', 'trader_data')

create_tables(conn)

data = collect_data(df, '2020-1-1', '2024-12-17')

upload(data, conn)

[*********************100%***********************]  503 of 503 completed


In [266]:
conn = sqlite3.connect('./data/trader_data.db')

query = """SELECT 
                 ticker
                ,ROUND(close, 2) as Close
                ,DATE(Date) AS Date
                ,GICSSector
                ,GICSSectorSub
                 CompanyName
            FROM equity 
            WHERE (1=1)
                AND Date > '2024-01-01'
                AND close IS NOT NULL"""

tableau_data = pd.read_sql_query(query, conn)

tableau_data.to_excel('2024_stock_data.xlsx')